In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg



MIN_MATCH_COUNT=8
# Best around 2000
detector = cv2.xfeatures2d.SIFT_create()
#print detector.hessianThreshold
#detector.upright = True
FLANN_INDEX_KDTREE=0
flannParam = dict(algorithm=FLANN_INDEX_KDTREE,tree=5)
flann = cv2.FlannBasedMatcher(flannParam,{})
font = cv2.FONT_HERSHEY_SIMPLEX

opY_axis =0
opX_axis =0
path = 'TrainingData'

#Code to load the training images
imagePaths = [os.path.join(path,f) for f in os.listdir(path)]
#print imagePaths
IDs = []
trainImageList = []
keyPointsList = []
descriptorList = []
imagePathList = []

for imagePath in imagePaths:
    trainImg=cv2.imread(imagePath,0)
    trainImageList.append(trainImg)
    imagePathList.append(imagePath)
    trainKP,trainDesc = detector.detectAndCompute(trainImg,None)
    keyPointsList.append(trainKP)
    descriptorList.append(trainDesc)
    imgId = os.path.split(imagePath)[-1].split('.')[0]
    IDs.append(imgId)
    
#Cam is the video capture object
cam=cv2.VideoCapture(0)  #0 is the default device number


#----- This is a main loop -------#

while True: 
    ret,QueryImgBGR=cam.read()    #Captures the frame in BGR, we have to convert it to a grayscale image
    #QueryImgBGR=cv2.resize(QueryImgBGR,(640,480))
    QueryImg=cv2.cvtColor(QueryImgBGR,cv2.COLOR_BGR2GRAY)    #Convert BGR to gray
    #Gets features from a QueryImg
    QueryKP,queryDesc=detector.detectAndCompute(QueryImg,None)
    
     #We will mach the QueryImg features with TrainImg features
    matchLengthList = []
    maxCountIndex = 0
    matchesList = []
    index = 0
    for trainDesc in descriptorList:
         
        matches=flann.knnMatch(queryDesc,trainDesc,k=2)
        goodMatch=[]
        for m,n in matches:
            if(m.distance<0.6*n.distance):
                goodMatch.append(m)
        if(len(goodMatch)>MIN_MATCH_COUNT):
            tp=[]
            qp=[]
            trainKP = keyPointsList[index]
            for m in goodMatch:
                tp.append(trainKP[m.trainIdx].pt)
                qp.append(QueryKP[m.queryIdx].pt)           
            tp,qp=np.float32((tp,qp))
            H,status=cv2.findHomography(tp,qp,cv2.RANSAC,5.0) 
            h,w=trainImageList[index].shape
            traningBorder=np.float32([[[0,0],[0,h-1],[w-1,h-1],[w-1,0]]])
            queryBorder=cv2.perspectiveTransform(traningBorder,H)
            opY_axis=queryBorder[0][1][1]
            opX_axis=queryBorder[0][1][0]
            print str(IDs[index])
            cv2.polylines(QueryImgBGR,[np.int32(queryBorder)],True,(0,255,0),5)
            cv2.putText(QueryImgBGR,str(IDs[index]),(opX_axis,opY_axis),font,4,(255,255,255),2,cv2.LINE_AA)
            trainImgShow=cv2.imread(imagePathList[index],1)
            #ims = cv2.resize(trainImgShow,(580,740))
            ims = cv2.resize(trainImgShow,(640,780))
            cv2.imshow('OUTPUT',trainImgShow)
        else:
            print "Not Enough match found- %d/%d"%(len(goodMatch),MIN_MATCH_COUNT)
        cv2.imshow('result',QueryImgBGR)
        if cv2.waitKey(10)==ord('q'):
            break
cam.release()
cv2.destroyAllWindows()
 
    





Not Enough match found- 1/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 1/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match found- 0/8
Not Enough match fou

IndexError: list index out of range